# Managed Agent Swarm - Policy Analysis

## Overview

This notebook explores a **managed agent swarm** pattern where a central orchestrator coordinates multiple specialist AI agents to collaboratively analyze farming policy documents. Unlike autonomous swarms where agents self-organize, or round-robin approaches where agents take predetermined turns, the managed swarm uses an intelligent orchestrator to dynamically direct the conversation based on the emerging discussion.

### The Experiment

The core idea being tested is whether an LLM-powered orchestrator can effectively facilitate a productive multi-agent dialogue by:
- Deciding which specialist to engage at each step
- Creating connections between agents' observations
- Directing follow-up questions that build on prior findings
- Knowing when sufficient insights have been gathered to synthesize a final report

### Use Case: Policy Document Analysis

We apply this pattern to analyze UK farming policy documents across three dimensions:
- **Critique Agent**: Analyzes how the document is written (clarity, structure, tone, accessibility)
- **Gap Analysis Agent**: Identifies missing information and coverage gaps
- **Ambiguity Agent**: Detects unclear language and vague requirements

The orchestrator facilitates a group discussion between these specialists, encouraging them to respond to each other's findings rather than working in isolation. This collaborative approach aims to surface insights that might be missed by sequential or independent analysis.

### Key Mechanisms

**Shared Message History**: All agents see the full conversation, enabling them to reference and build on each other's observations.

**Dynamic Tool Selection**: The orchestrator chooses which specialist to engage based on the current state of the discussion, not a fixed sequence.

**Human-in-the-Loop Handoff**: When the orchestrator determines sufficient analysis has been completed, it hands off to a report writer agent. A human approval step allows for feedback or acceptance before final report generation.

**Usage Limits**: Token and tool call limits prevent runaway execution while encouraging focused, efficient analysis.

---

For environment setup and prerequisites, see the [README.md](../../README.md) in the repository root.

In [ ]:
import os

import dotenv

dotenv.load_dotenv(dotenv_path="../.env")

## Environment Setup

Configuration for the notebook environment. We use `python-dotenv` to load environment variables from `../notebooks/.env`, which contains AWS credentials for Bedrock access and optional OpenTelemetry configuration.

See the [README.md](../../README.md) for details on required environment variables.

In [ ]:
import pydantic_ai
from pydantic_ai.models import bedrock as bedrock_models

claude_haiku = bedrock_models.BedrockConverseModel(
    model_name="anthropic.claude-3-haiku-20240307-v1:0"
)

claude_sonnet = bedrock_models.BedrockConverseModel(
    model_name="anthropic.claude-3-sonnet-20240229-v1:0"
)

bedrock_model_settings = bedrock_models.BedrockModelSettings(
    bedrock_guardrail_config={"trace": "enabled"}
)

## Model Configuration

We configure two Claude models from Amazon Bedrock via the `pydantic-ai` library:

- **Claude Haiku**: Fast, lightweight model used for the orchestrator and specialist agents. Ideal for quick analysis and coordination tasks.
- **Claude Sonnet**: More powerful model reserved for the report writer agent, which needs to synthesize complex information into a cohesive final report.

The `BedrockModelSettings` is shown here to demonstrate that Bedrock clients can be customised as needed for the application of guardrails, logging, or other settings.

In [ ]:
from dataclasses import dataclass, field

from pydantic_ai.messages import (
    ModelMessage,
    ModelRequest,
    ModelResponse,
    TextPart,
    UserPromptPart,
)


@dataclass
class AgentDependencies:
    source_policy: str
    messages: list[ModelMessage] = field(default_factory=list)

## Agent Dependencies

The `AgentDependencies` dataclass defines shared state that flows through all agent interactions:

- **source_policy**: The farming policy document text being analyzed
- **messages**: Shared conversation history that enables agents to see and reference prior observations

Pydantic AI recommends using a dependency injection pattern to manage shared state across agents,

## OpenTelemetry Tracing Setup

Optional observability configuration using OpenTelemetry to trace agent interactions. When `OTEL_EXPORTER_OTLP_ENDPOINT` is set, this sends detailed execution traces (spans) to an OTLP collector.

Pydantic AI has built-in support for OpenTelemetry tracing of agent operations. This could be useful if we get asked to demonstrate or debug the multi-agent interactions in more detail.

In [ ]:
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.trace import set_tracer_provider


def setup_tracing():
    if not os.getenv("OTEL_EXPORTER_OTLP_ENDPOINT"):
        print("OTEL_EXPORTER_OTLP_ENDPOINT not set, skipping tracing setup")
        return

    os.environ["OTEL_SERVICE_NAME"] = "managed-agent-swarm"

    exporter = OTLPSpanExporter()
    span_processor = BatchSpanProcessor(exporter)
    tracer_provider = TracerProvider()
    tracer_provider.add_span_processor(span_processor)

    set_tracer_provider(tracer_provider)
    return


setup_tracing()

## Swarm Architecture Diagram

This Mermaid flowchart visualizes the managed swarm architecture:

- **Orchestrator**: The central coordinator that maintains conversation history and decides which specialist to engage
- **Three Specialist Agents**: Critique, Gap Analysis, and Ambiguity agents that provide domain-specific analysis
- **Bidirectional Communication**: The orchestrator can ask questions and receive responses from any specialist based on discussion needs
- **Human Approval Gate**: When the orchestrator believes sufficient insights have been gathered, it requests human review before proceeding
- **Two-Path Decision**: 
  - **Approve**: Human accepts the analysis quality and proceeds to report generation
  - **Feedback**: Human provides guidance to continue/refine the discussion (shown as dotted line back to orchestrator)
- **Report Writer**: Synthesizes the approved conversation into a structured final report
- **Final Report**: The comprehensive output with executive summary, findings, analysis, and recommendations

The key insight is that the orchestrator dynamically manages the conversation flow rather than following a predetermined sequence, while the human approval step ensures quality control before finalizing results.

```mermaid
flowchart LR
    User[User] --> Orch[Orchestrator<br/>+ Message History]
    
    Orch <-->|ask/respond| Critique[Critique<br/>Agent]
    Orch <-->|ask/respond| Gap[Gap<br/>Agent]
    Orch <-->|ask/respond| Ambig[Ambiguity<br/>Agent]
    
    Orch ==>|ready for handoff| Human{Human<br/>Approval}
    Human -->|approve| Writer[Report<br/>Writer]
    Human -.->|feedback| Orch
    Writer --> Report[Final<br/>Report]
    
    style Orch fill:#e1f5ff,stroke:#0066cc,stroke-width:3px
    style Human fill:#fff3cd,stroke:#ff9800,stroke-width:3px
    style Writer fill:#e8f5e9,stroke:#4caf50,stroke-width:2px
    style Report fill:#f3e5f5,stroke:#9c27b0,stroke-width:2px
```

## Agent Definitions

### Orchestrator Agent

The orchestrator is the "swarm manager" with several key responsibilities:

1. **Tool-based coordination**: Uses three tool functions (`ask_critique_agent`, `ask_gap_analysis_agent`, `ask_ambiguity_agent`) to engage specialists
2. **Dynamic decision-making**: Chooses which specialist to call based on conversation state, not a fixed pattern
3. **Dialogue facilitation**: System prompt instructs it to create connections between agents' observations
4. **Handoff decision**: Determines when sufficient analysis has occurred and triggers report generation

The orchestrator's output type is set to `handoff_to_report_writer`, making handoff the natural conclusion of its execution.

### Specialist Agents

Each specialist agent has a focused role with a system prompt designed to encourage collaborative dialogue:

**Critique Agent**: Analyzes document writing quality - clarity, structure, tone, and accessibility. Prompted to reference other agents' observations when responding.

**Gap Analysis Agent**: Identifies missing information and coverage gaps. Encouraged to connect gaps to issues raised by other agents.

**Ambiguity Agent**: Detects unclear or vague language. Prompted to explain how ambiguities might cause problems identified by other specialists.

All three specialist agents share:
- The same model (Claude Haiku) for consistency
- Access to shared message history for context
- System prompts emphasizing dialogue and cross-referencing
- Tools to query the source policy document

### Tool Functions

Each tool function (`ask_*_agent`) handles the interaction with a specialist:

1. **Logs the question** being asked (for visibility during execution)
2. **Calls the specialist agent** with the source policy and question, passing shared message history
3. **Records the interaction** in the shared message history as a request/response pair
4. **Returns formatted response** to the orchestrator

This pattern ensures the conversation is properly tracked and available to all agents.

### Output Models

**FinalReport**: A Pydantic model defining the structure of the final analysis with fields for executive summary, key findings, detailed analysis, and recommendations.

**ReportRejected**: A signal model indicating human feedback was provided instead of approval, prompting continued discussion.

In [ ]:
from pydantic import BaseModel


# Final report model
class FinalReport(BaseModel):
    """Final comprehensive report"""

    executive_summary: str
    key_findings: str
    detailed_analysis: str
    recommendations: str

    def __repr__(self) -> str:
        return (
            f"FinalReport(\n"
            f"  Executive Summary: {self.executive_summary}\n"
            f"  Key Findings: {self.key_findings}\n"
            f"  Detailed Analysis: {self.detailed_analysis}\n"
            f"  Recommendations: {self.recommendations}\n"
            f")"
        )


class ReportRejected(BaseModel):
    """Indicates that the report was rejected and discussion should continue"""


instrumentation_settings = pydantic_ai.InstrumentationSettings(
    include_binary_content=False,
    include_content=True,
)

critique_agent = pydantic_ai.Agent(
    model=claude_haiku,
    model_settings=bedrock_model_settings,
    deps_type=AgentDependencies,
    output_type=str,
    system_prompt="""You are a critique specialist that analyzes how farming policy documents are written.
Focus on clarity, structure, tone, accessibility, and communication effectiveness.

IMPORTANT: You are part of an active group discussion. Review ALL previous messages carefully:

When responding to questions:
- ALWAYS acknowledge and reference specific points made by other agents
- Build directly on their findings with your unique perspective
- Challenge or validate their observations explicitly
- Ask follow-up questions when you need clarity from them
- Point out connections between your analysis and theirs

Examples:
- "I agree with Gap Analysis Agent's point about X. The missing content creates structural issues because..."
- "Ambiguity Agent, you mentioned Y was unclear. I've noticed this also affects the document flow in section Z..."
- "Building on what Gap Analysis found, the lack of X makes the tone inconsistent when..."

Be conversational and engaged. This is a dialogue, not a report.

Use all the tools at your disposal to reference the target policy document as needed.""",
    instrument=instrumentation_settings,
)

gap_analysis_agent = pydantic_ai.Agent(
    model=claude_haiku,
    model_settings=bedrock_model_settings,
    deps_type=AgentDependencies,
    output_type=str,
    system_prompt="""You are a gap analysis specialist for farming policies.
Identify missing information, overlooked areas, and gaps in coverage.
Consider what's not addressed that should be for comprehensive policy.

IMPORTANT: You are part of an active group discussion. Review ALL previous messages carefully:

When responding to questions:
- ALWAYS reference specific observations from other agents
- Explain how gaps you find relate to issues they've raised
- Directly address their concerns with evidence from the policy
- Ask them for clarification or additional perspective
- Build collaborative understanding

Examples:
- "Critique Agent, your concern about structure is spot-on. The document is missing..."
- "Responding to Ambiguity Agent's point about vague language - I see the same pattern in missing definitions..."
- "This gap I found might explain why Critique Agent observed X. Can we explore that connection?"

Engage actively with the conversation. Reference others' names and specific points.

Use all the tools at your disposal to reference the target policy document as needed.""",
    instrument=instrumentation_settings,
)

ambiguity_agent = pydantic_ai.Agent(
    model=claude_haiku,
    model_settings=bedrock_model_settings,
    deps_type=AgentDependencies,
    output_type=str,
    system_prompt="""You are an ambiguity detection specialist.
Identify unclear language, vague requirements, and areas open to interpretation.
Highlight potential confusion points and suggest clarifications.

IMPORTANT: You are part of an active group discussion. Review ALL previous messages carefully:

When responding to questions:
- ALWAYS connect your findings to what other agents have said
- Explain how vague language might be causing the issues they identified
- Directly address their specific concerns
- Ask for their input on whether ambiguities explain their observations
- Create dialogue by referencing their names and points

Examples:
- "Gap Analysis Agent found X is missing. I wonder if that's because the language around it is so vague..."
- "Critique Agent, the structure issue you noted - could it stem from this ambiguous terminology?"
- "I agree with both of you. The vagueness I'm seeing in section Y connects to both the gaps and structure problems..."

Be conversational and collaborative. Make this a real discussion.

Use all the tools at your disposal to reference the target policy document as needed.""",
    instrument=instrumentation_settings,
)

report_writer_agent = pydantic_ai.Agent(
    model=claude_sonnet,
    model_settings=bedrock_model_settings,
    deps_type=AgentDependencies,
    output_type=str,
    system_prompt="""You are a report writing specialist.
Synthesize all the group discussion into a comprehensive, well-structured report.
Review the entire conversation history and create a cohesive analysis.

Use all the tools at your disposal to reference the target policy document as needed.""",
    instrument=instrumentation_settings,
)


# Output function for handoff to report writer with human approval
async def handoff_to_report_writer(
    ctx: pydantic_ai.RunContext[AgentDependencies],
) -> FinalReport | ReportRejected:
    """Hand off to the report writer to synthesize all discussion into a final report.
    Use this when the group discussion has produced sufficient insights."""

    print("\n" + "=" * 80)
    print("🎯 ORCHESTRATOR READY FOR REVIEW")
    print("=" * 80)
    print("\nThe orchestrator has completed the group discussion.\n")
    print(f"Total messages in conversation: {len(ctx.deps.messages)}\n")

    # Show summary of conversation
    print("📋 CONVERSATION SUMMARY:")
    print(ctx.deps.messages[-10:])  # Show last 10 messages for brevity

    decision = input(
        "Do you approve generating the final report? (yes/approve to proceed, any other input to provide feedback): "
    ).strip()

    if decision.lower() in ["yes", "approve", "y"]:
        print("\n✅ Approved! Generating final report...\n")
        print("🔄 Handing off to Report Writer Agent...")

        # Pass the conversation history to the report writer
        response = await report_writer_agent.run(
            user_prompt="Review the entire conversation history and create a comprehensive report synthesizing all findings.",
            deps=ctx.deps,
            output_type=FinalReport,
            message_history=ctx.deps.messages,
        )

        print("✅ Report Writer completed\n")
        return response.output
    feedback = input("Please provide your feedback for the agents: ").strip()
    print("\n🔄 Feedback received. Continuing discussion...\n")
    # Return feedback as string (not a FinalReport)
    # This will be caught in the execution loop
    return f"[User]: Output Rejected - {feedback}"


# Orchestrator agent with tools to coordinate the group chat
orchestrator_agent = pydantic_ai.Agent(
    model=claude_haiku,
    model_settings=bedrock_model_settings,
    deps_type=AgentDependencies,
    output_type=handoff_to_report_writer,
    instrument=instrumentation_settings,
    system_prompt="""You are an orchestrator coordinating a group discussion about farming policy analysis.

You have three specialist agents available:
- Critique Agent: analyzes how the document is written
- Gap Analysis Agent: identifies missing information and coverage gaps
- Ambiguity Agent: finds unclear or vague language

Your role is to FACILITATE dialogue between the agents:

1. Start by getting perspectives from the agents who are most relevant to the analysis
2. When an agent makes an observation, consider whether other agents should respond:
   - Ask agents to build on each other's findings
   - Direct follow-up questions when connections emerge
   - Example: "Ambiguity Agent, Critique Agent mentioned unclear structure. Does vague language contribute to this?"
3. Create natural back-and-forth by having agents respond to each other's specific points
4. Push for concrete examples when agents make general observations

IMPORTANT PRINCIPLES:
- ALWAYS include the agent's name in your question
- Reference specific points from other agents when creating dialogue
- Make agents talk TO each other, not just to you
- If an agent doesn't reference others, consider asking them to engage with prior observations

When you feel the discussion has produced sufficient insights to answer the analysis comprehensively, hand off to the report writer. Don't force unnecessary rounds - quality over quantity.""",
)


@orchestrator_agent.tool
async def ask_critique_agent(
    ctx: pydantic_ai.RunContext[AgentDependencies], question: str
) -> str:
    """Ask the critique agent to analyze how the document is written."""

    print(f"\n📝 Critique Agent: {question[:100]}...")

    response = await critique_agent.run(
        user_prompt=[ctx.deps.source_policy, question],
        deps=ctx.deps,
        message_history=ctx.deps.messages,
    )

    # Add to message history as a request/response pair
    ctx.deps.messages.append(
        ModelRequest(
            parts=[UserPromptPart(content=f"[Critique Agent] {question}")],
            timestamp=None,
        )
    )

    formatted = f"[Critique Agent] {response.output}"

    ctx.deps.messages.append(
        ModelResponse(parts=[TextPart(content=formatted)], timestamp=None)
    )

    print(f"✅ Response received ({len(response.output)} chars)\n")
    return formatted


@orchestrator_agent.tool
async def ask_gap_analysis_agent(
    ctx: pydantic_ai.RunContext[AgentDependencies], question: str
) -> str:
    """Ask the gap analysis agent to identify missing information or coverage gaps."""

    print(f"\n🔍 Gap Analysis Agent: {question[:100]}...")

    response = await gap_analysis_agent.run(
        user_prompt=[ctx.deps.source_policy, question],
        deps=ctx.deps,
        message_history=ctx.deps.messages,
    )

    # Add to message history as a request/response pair
    ctx.deps.messages.append(
        ModelRequest(
            parts=[UserPromptPart(content=f"[Gap Analysis Agent] {question}")],
            timestamp=None,
        )
    )

    formatted = f"[Gap Analysis Agent] {response.output}"

    ctx.deps.messages.append(
        ModelResponse(parts=[TextPart(content=formatted)], timestamp=None)
    )

    print(f"✅ Response received ({len(response.output)} chars)\n")
    return formatted


@orchestrator_agent.tool
async def ask_ambiguity_agent(
    ctx: pydantic_ai.RunContext[AgentDependencies], question: str
) -> str:
    """Ask the ambiguity agent to identify unclear or vague language."""

    print(f"\n❓ Ambiguity Agent: {question[:100]}...")

    response = await ambiguity_agent.run(
        user_prompt=[ctx.deps.source_policy, question],
        deps=ctx.deps,
        message_history=ctx.deps.messages,
    )

    # Add to message history as a request/response pair
    ctx.deps.messages.append(
        ModelRequest(
            parts=[UserPromptPart(content=f"[Ambiguity Agent] {question}")],
            timestamp=None,
        )
    )

    formatted = f"[Ambiguity Agent] {response.output}"

    ctx.deps.messages.append(
        ModelResponse(parts=[TextPart(content=formatted)], timestamp=None)
    )

    print(f"✅ Response received ({len(response.output)} chars)\n")
    return formatted

@orchestrator_agent.system_prompt
def system_prompt() -> str:
    return """You are an orchestrator coordinating a group discussion about farming policy analysis."""

## Execution

### Policy Selection

Select which farming policy document to analyze by setting the `policy_name` variable. The policy content is loaded from markdown files scraped by the `farming-funding.ipynb` notebook.

The `AgentDependencies` object is initialized with the policy text and an empty message history.

### Orchestration Loop

The orchestrator is executed with a prompt to facilitate comprehensive analysis. Key execution parameters:

- **UsageLimits**: Each orchestrator run gets 25 tool calls to prevent runaway execution. Note that if the human provides feedback and the loop continues, the orchestrator gets a fresh 25 tool calls for the next iteration.
- **Message history**: Maintains conversation continuity across all runs, even when usage limits reset

**The handoff function** (`handoff_to_report_writer`) provides human-in-the-loop control:
1. Displays conversation summary when orchestrator is ready for handoff
2. Prompts human to approve or provide feedback
3. On approval: Calls report writer agent to synthesize findings
4. On rejection: Returns feedback string to continue discussion

**The continuation loop** checks if the output is a `FinalReport`:
- If not (feedback was provided), it continues the orchestrator run with the feedback and fresh usage limits
- If yes, the loop exits and the final report is available

This pattern ensures humans can guide the analysis quality and depth before finalizing results, with the ability to run multiple rounds of analysis if needed.

In [ ]:
import pathlib

policy_name = "AGF1"
policy_path = pathlib.Path(f"../scrapers/outputs/{policy_name.upper()}.md")

# Initialize dependencies
deps = AgentDependencies(source_policy=policy_path.read_text())

operation_usage: list[pydantic_ai.RunUsage] = []

# Run the orchestrator to coordinate the group chat
orchestrator_response = await orchestrator_agent.run(
    user_prompt=[
        """Facilitate a group discussion to comprehensively analyze this farming policy document.
    Let the agents respond to each other's findings and build on the discussion.
    When you've gathered sufficient insights (aim for thorough coverage), generate the final report.""",
        deps.source_policy,
    ],
    deps=deps,
    usage_limits=pydantic_ai.UsageLimits(tool_calls_limit=25),
)

operation_usage.append(orchestrator_response.usage)

while not isinstance(orchestrator_response.output, FinalReport):
    feedback = orchestrator_response.output
    print(f"\n🔄 Continuing discussion based on feedback:\n{feedback}\n")

    # Continue the orchestrator run with the feedback
    orchestrator_response = await orchestrator_agent.run(
        user_prompt=[
            deps.source_policy,
            f"{feedback}\n\nContinue the group discussion to address the feedback and work towards a final report.",
        ],
        deps=deps,
        usage_limits=pydantic_ai.UsageLimits(tool_calls_limit=25),
    )

    operation_usage.append(orchestrator_response.usage)

## Results

### View Final Report

Display the final synthesized report if one was generated. The report follows the `FinalReport` model structure with executive summary, key findings, detailed analysis, and recommendations.

If the execution was stopped before handoff approval, no report will be available.

In [ ]:
import pprint

orchestrator_response.output

### View Complete Trace

Optional diagnostic output showing the full message history and tool calls in JSON format. Useful for:
- Understanding the orchestrator's decision-making process
- Debugging unexpected behavior
- Analyzing conversation flow patterns
- Evaluating agent interaction quality

This provides the raw execution trace that can be correlated with OpenTelemetry spans if tracing is enabled.

In [ ]:
# Optional: View the complete message history and tool calls
print("=== COMPLETE MESSAGE TRACE ===\n")
pprint.pprint(str(orchestrator_response.all_messages_json()))